In [332]:
import xpress as xp
import numpy as np
import pandas as pd

In [333]:
# read date info from csv file
df = pd.read_csv('final_optimized_future_season_schedule.csv')

# colomn: 1 is day, 2,3,4 are F,M,A
F = df.iloc[:,1].values
M = df.iloc[:,2].values
A = df.iloc[:,3].values

for i in range(len(A)):
    if A[i] == 1:
        A[i] = i
    else:
        A[i] = -1
    if M[i] == 1:
        M[i] = i
    else:
        M[i] = -1
    if F[i] == 1:
        F[i] = i
    else:
        F[i] = -1

F = F[F != -1]
M = M[M != -1]
A = A[A != -1]

print(F)
print(M)
print(A)

[ 0  3  4  5  8  9 10 11 13 14 19 20 22 23 24 26 27 29 32 38 40 41 44 47
 53 54]
[ 2 31 34 36 43 45]
[48 49 50]


In [ ]:
nT = 11
nN = 6
nD = 56
nTotalGames = 42
nHalfGames = 21
nSouthGames = 6
nSouthHalf = 3
nNorthIn = 6
nNorthOut = 5
nHalfIn = 3
nHalfOut = 2
nHalfOuts = 3
nSouthNorth = 3
nSNHalf = 1
nSNHalfs = 2

In [335]:
# Sets

# 00#FrontRoyal 01#NewMarket 02#Purcellville
# 03#Strasburg 04#Winchester 05#Woodstock
# 06#Charlottesville 07#Covington
# 08#Harrisonburg 09#Staunton 10#Waynesboro
Team = ["FrontRoyal", "NewMarket", "Purcellville", "Strasburg", "Winchester", "Woodstock", "Charlottesville", "Covington", "Harrisonburg", "Staunton", "Waynesboro"]

# T: all teams in the league
T = np.arange(nT)

# N: teams in the North Division
N = np.arange(0, nN)

# S: teams in the South Division
S = np.arange(nN, nT)

# G1: North Division teams that play more frequently
G1 = np.arange(0, nN//2)

# G2: North Division teams that play more frequently, where G1 ∩ G2  ∅
G2 = np.arange(nN//2, nN)

# D: days during the season
D = np.arange(nD)

# W: weekend (Friday and Saturday) days during the season
W = np.array([0,1, 7,8, 14,15, 21,22, 28,29, 35,36, 42,43, 49,50])

# P: prime days during the season, Fridays and Saturdays and the last three days of the season
P = np.concatenate([W, np.array([53, 54, 55])])

# Su: Sundays during the season
Su = np.array([2, 9, 16, 23, 30, 37, 44, 51])

# F: days during the season that require a full slate of games to be played

# M: strategic idle days across the entire league

# A: days of the All-Star break

# 33: days when Winchester does not want to play on the road against Covington or Charlottesville
i33 = np.array([4])
j33 = np.array([6, 7])
k33 = np.array([25, 26, 27, 28, 39, 40, 41, 42])

# 34: day when Waynesboro travels to play in Charlottesville
i34 = 6
j34 = 10
k34 = 36

# K: days of the dth week of the season, for d  1,:::,|D|/=7
K = np.array([np.arange(7 * i, 7 * (i + 1)) for i in range(nD//7)])

WEEK = np.array([0, 1, 2, 3, 4, 5, 6, 7])
ONE = np.array([1])
TWO = np.array([2])
TEN = np.arange(11)
minusONE = np.arange(0, nD - 1)
minusTWO = np.arange(0, nD - 2)
minusTEN = np.arange(0, nD - 10)


In [336]:
# Parameters
#gd: minimum number of games each team must play during the d th week of the season, for d  1,:::,|D|=7
Gmin = [6, 6, 4, 4, 3, 3, 4, 4]

# δij: equals 1 if team i and team j are in the same division and 0 otherwise
Same = [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]]

# γik: equals 1 if team i cannot play a home game on day k and 0 otherwise
Yhome = np.zeros((11, 56)) # TBD

# dij: rank order of travel distance in miles for each team i to play at opponent j,
# where rank 1 indicates the farthest opponent of team i
Dis = np.array([[99, 6, 7, 10, 9, 8, 4, 1, 5, 2, 3],
              [7, 99, 2, 8, 3, 10, 4, 1, 9, 6, 5],
              [9, 6, 99, 8, 10, 7, 4, 1, 5, 2, 3],
              [10, 7, 6, 99, 8, 9, 4, 1, 5, 3, 2],
              [10, 6, 8, 9, 99, 7, 4, 1, 5, 3, 2],
              [8, 9, 5, 10, 7, 99, 4, 1, 6, 3, 2],
              [5, 7, 1, 4, 3, 6, 99, 2, 8, 9, 10],
              [3, 6, 1, 4, 2, 5, 7, 99, 8, 10, 9],
              [4, 10, 1, 5, 3, 7, 6, 2, 99, 9, 8],
              [3, 7, 1, 4, 2, 5, 8, 6, 9, 99, 10],
              [3, 7, 1, 4, 2, 6, 9, 5, 8, 10, 99]])

In [337]:
# Decision variables

# Xijk = 1 if team i is home to team j on day k
X = {
    (i, j, k): xp.var(name=f"X_{i}_{j}_{k}", vartype=xp.binary)
    for i in T for j in T for k in D
}

In [338]:
# Objective

model = xp.problem()

model.addVariable(X)

In [339]:
# Constraints
# A.1
# Objective function X[i, j, k] * Same[i, j]
# A.2
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for j in T for k in D) == nTotalGames for i in T)
# A.3
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in D) == nHalfGames for i in T)
# A.4
model.addConstraint(xp.Sum(X[i, i, k] for k in D) == 0 for i in T)
# A.5
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for j in T) <= 1 for i in T for k in D)

In [340]:
# A.6
# except Charlottesville and Waynesboro, which have fewer because they play a Fourth of July game ???
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in W) >= 7 for i in T)
# A.7
# F is opening day or before a league-wide holiday ???
model.addConstraint(xp.Sum(X[i, j, k] for i in T for j in T) >= 5 for k in F)
# # A.8
# MA = np.unique(np.concatenate((M, A)))
# model.addConstraint(xp.Sum(X[i, j, k] for i in T for j in T for k in MA) == 0)
# A.9
model.addConstraint(xp.Sum(X[i, j, k] for j in T) == 0 for i in T for k in D if Yhome[i, k] == 1)

In [341]:
# A.10
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for k in D) == nSouthGames for i in S for j in S if i != j)
# A.11
model.addConstraint(xp.Sum(X[i, j, k] for k in D) == nSouthHalf for i in S for j in S if i != j)
# A.12
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for k in D) == nNorthIn for i in G1 for j in G1 if i != j)
# A.13
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for k in D) == nNorthIn for i in G2 for j in G2 if i != j)
# A.14
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for k in D) == nNorthOut for i in G2 for j in G1)
# A.15
model.addConstraint(xp.Sum(X[i, j, k] for k in D) == nHalfIn for i in G1 for j in G1 if i != j)
# A.16
model.addConstraint(xp.Sum(X[i, j, k] for k in D) == nHalfIn for i in G2 for j in G2 if i != j)
# A.17
model.addConstraint(xp.Sum(X[i, j, k] for k in D) >= nHalfOut for i in G2 for j in G1)
model.addConstraint(xp.Sum(X[i, j, k] for k in D) <= nHalfOuts for i in G2 for j in G1)
# A.18
model.addConstraint(xp.Sum(X[i, j, k] for k in D) >= nHalfOut for i in G1 for j in G2)
model.addConstraint(xp.Sum(X[i, j, k] for k in D) <= nHalfOuts for i in G1 for j in G2)
# A.19
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for k in D) == nSouthNorth for i in S for j in N)
# A.20
model.addConstraint(xp.Sum(X[i, j, k] for k in D) >= nSNHalf for i in S for j in N)
model.addConstraint(xp.Sum(X[i, j, k] for k in D) <= nSNHalfs for i in S for j in N)

In [342]:
# from 92 to 73
# A.21
model.addConstraint(xp.Sum(X[i, j, z+a] + X[j, i, z+a] for j in T for a in TEN) <= 10 for i in T for z in minusTEN)
# # A.22 ZZZZZZZZZZZZZZZZZZZZZZZZZ ???
# model.addConstraint(xp.Sum(X[i, j, z+a] + X[j, i, z+a] for j in T for a in ONE) >= 1 for i in T for z in M)
# A.23
model.addConstraint(xp.Sum(X[i, j, z+a] for j in T for a in TWO) <= 2 for i in T for z in minusTWO)
# A.24
model.addConstraint(xp.Sum(X[i, j, z+a] for j in T for a in TWO) <= 2 for i in T for z in minusTWO)
# A.25 MMMMMMMMMMMMMMMMMMMMM[55]
model.addConstraint(xp.Sum(X[i, j, z-1] + X[j, i, z-1] + X[i, j, z+1] + X[j, i, z+1] for j in T) >= 1 for i in T for z in M)
# A.26
model.addConstraint(xp.Sum(X[i, j, A[0]-1] + X[j, i, A[0]-1] + X[i, j, A[0]+2] + X[j, i, A[0]+2] for j in T) >= 1 for i in T)

In [343]:
# A.27
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in K[d]) >= 2 for i in T for d in WEEK)
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in K[d]) <= 4 for i in T for d in WEEK)
# A.28
model.addConstraint(xp.Sum(X[j, i, k] for j in T for k in K[d]) >= 2 for i in T for d in WEEK)
model.addConstraint(xp.Sum(X[j, i, k] for j in T for k in K[d]) <= 4 for i in T for d in WEEK)
# # A.29
# model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in K[d]) >= Gmin[d] for i in T for d in WEEK)
# A.30
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in Su) >= 3 for i in T)
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in Su) <= 4 for i in T)
# A.31
model.addConstraint(xp.Sum(X[i, j, z+a] + X[j, i, z+a] for a in TWO) <= 1 for i in T for j in T for z in minusTWO)
# A.32
model.addConstraint(xp.Sum(X[j, i, z+a] for a in ONE for j in T if Dis[i, j] <= 3) <= 1 for i in T for z in minusONE)

In [344]:
# A.33
model.addConstraint(xp.Sum(X[j, i, k] for i in i33 for j in j33 for k in k33) == 0)
# A.34
model.addConstraint(X[i34, j34, k34] == 1)
# A.35
# X[i, j, k] is binary for i in T, j in T, k in D

In [345]:
# Objective function
MMAX = xp.Sum((X[i, j, k] * Same[i][j]) for i in T for j in T for k in P)

model.setObjective(MMAX, sense=xp.maximize)
model.setControl("maxtime", 10)
model.solve()

FICO Xpress v9.4.2, Hyper, solve started 11:56:31, Feb 18, 2025
Heap usage: 10MB (peak 10MB, 270KB system)
Maximizing MILP noname using up to 16 threads and up to 27GB memory, with these control settings:
MAXTIME = 10
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
     10171 rows         6776 cols       230000 elements      6776 entities
Presolved problem has:
      2154 rows         6106 cols       166390 elements      6106 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 13MB (peak 22MB, 270KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  2.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  4.20e+01] / [ 1.00e+00,  1.50e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

Symmetric problem: generators: 4, support set: 5460
 Number of orbi

(<SolveStatus.STOPPED: 1>, <SolStatus.FEASIBLE: 2>)

In [346]:
print(f"Total Games: {model.getObjVal()}")

TT = 0

for week in range(nD//7):
    for i in T:
        for k in range(7):
            for j in T:
                if model.getSolution(X[i, j, k+7*week]) > 0.5:
                    print(1, end="")
                    TT += 1
                else:
                    print('.', end="")
            print("  ", end="")
        print()
    print()

print(TT)

    

Total Games: 73.0
...1.......  ....1......  ...........  ...........  ...........  ...........  ...........  
...........  ...........  1..........  .........1.  ...........  ...........  ...........  
...........  ...........  .........1.  ...........  1..........  ...........  ...........  
...........  ........1..  ...........  ...........  ......1....  ...........  .......1...  
..1........  ...........  .......1...  ...........  ...........  1..........  ...........  
...........  ..1........  ..........1  ...1.......  ...........  ...........  ...........  
...........  .1.........  ...........  ...........  ...........  .....1.....  ...........  
...........  ...........  ...........  ..1........  .....1.....  ...1.......  ...........  
.1.........  ...........  ...........  ......1....  ...........  ..........1  ...........  
......1....  ...........  ...........  ...........  .1.........  ..1........  ...........  
.....1.....  ...........  ...........  ....1......  ........1.